In [119]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
from tensorflow.keras.preprocessing.text import Tokenizer
from sklearn.model_selection import KFold
from tensorflow.keras.layers import Conv2D
from tensorflow.keras.layers import Concatenate
from tensorflow.keras.layers import concatenate
from tensorflow import keras
from tensorflow.keras.layers import Bidirectional
from tensorflow.keras.models import Model
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Flatten, LSTM, Conv1D, MaxPooling1D, Dropout, Activation
from tensorflow.python.keras.layers.embeddings import Embedding
from sklearn.model_selection import train_test_split
from sklearn.metrics import f1_score
from sklearn.preprocessing import MinMaxScaler
import nltk
import string
from nltk.corpus import stopwords
import re
from nltk.stem.snowball import SnowballStemmer
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Flatten, LSTM, Conv1D, MaxPooling1D, Dropout, Activation

In [120]:
train_df=pd.read_csv('train_limpio_sin_vect_textos.csv')
train_tfidf = pd.read_csv('train_TfIdf_Dimensiones_Reducidads.csv')
test_df=pd.read_csv('test_limpio_sin_vect_textos.csv')
test_tfidf = pd.read_csv('test_TfIdf_Dimensiones_Reducidas.csv')
sample = pd.read_csv("sample_submission.csv",encoding = "ISO-8859-1")

In [121]:
def clean_text(text):
    
    ## Remove puncuation
    text = text.translate(string.punctuation)
    ## Convert words to lower case and split them
    text = text.lower()
       
    ## Remove stop words (mejor las dejo)
    #text = text.split()
    #stops = set(stopwords.words("english"))
    #text = [w for w in text if not w in stops and len(w) >= 3]  
    #text = " ".join(text)

    ## Clean the text
    text = re.sub('[%s]' % re.escape(string.punctuation), '', text)  #quito caracteres inusuales
    text = re.sub('â' , '', text)  #quito caracteres inusuales
    text = re.sub('ª' , '', text)  #quito caracteres inusuales
    text = re.sub('ã' , '', text)  #quito caracteres inusuales
    text = re.sub('¼' , '', text)  #quito caracteres inusuales
    text = re.sub('\w*\d\w*', '', text)  # quito numeros
    text = re.sub("[^0-9a-zA-Z]+", " ", text)
    
    ## Stemming
    #text = text.split()
    #stemmer = SnowballStemmer('english')
    #stemmed_words = [stemmer.stem(word) for word in text]
    #text = " ".join(stemmed_words)

    return text

In [122]:
train_df['text'] = train_df['text'].replace(r'http\S+', '', regex=True)
test_df['text'] = test_df['text'].replace(r'http\S+', '', regex=True)
train_df['text'] = train_df['text'].apply(lambda x: str(x).lower())
test_df['text'] = test_df['text'].apply(lambda x: str(x).lower())
target = train_df.target

Preparo el texto para el embedding

In [123]:
#Esto no deberia ser necesario
train_df.text=train_df.text.map(lambda x: clean_text(x))
test_df.text=test_df.text.map(lambda x: clean_text(x))

In [124]:
tokenizer = Tokenizer()
tokenizer.fit_on_texts(train_df['text'])

train_sequences = tokenizer.texts_to_sequences(train_df['text'])
train_data = pad_sequences(train_sequences, maxlen=180)

In [125]:
test_sequences = tokenizer.texts_to_sequences(test_df['text'])
test_data = pad_sequences(test_sequences, maxlen=180)

In [126]:
EMBEDDING_FILE = 'glove-dataset/glove.twitter.27B.200d.txt' 
#Por ahora lo dejoe n 50, supongo que solo mejora con 200 si complejizo la red nueronal
def get_coefs(word,*arr): return word, np.asarray(arr, dtype='float32')
embeddings_index = dict(get_coefs(*o.split(" ")) for o in open(EMBEDDING_FILE, 'r', encoding='utf-8'))

In [127]:
# create a weight matrix for words in training docs
embedding_matrix = np.zeros((50000, 200)) #50 o 200, segun las dimensiones del objeto que use
for word, index in tokenizer.word_index.items():
    if index > 50000 - 1:
        break
    else:
        embedding_vector = embeddings_index.get(word)
        if embedding_vector is not None:
            embedding_matrix[index] = embedding_vector

Preparo el input de las keywords para la Red Neuronal

In [128]:
train_df['keyword'] = train_df['keyword'].apply(lambda x: str(x).lower())
test_df['keyword'] = test_df['keyword'].apply(lambda x: str(x).lower())

In [129]:
tokenizer = Tokenizer()
tokenizer.fit_on_texts(train_df['keyword'])

train_sequences = tokenizer.texts_to_sequences(train_df['keyword'])
train_keyword_data = pad_sequences(train_sequences, maxlen=50)

In [130]:
test_sequences = tokenizer.texts_to_sequences(test_df['keyword'])
test_keyword_data = pad_sequences(test_sequences, maxlen=50)

Preparo el input de las demas features

In [154]:
train_no_text = train_df.drop(columns=['text','keyword','target']).join(train_tfidf)
test_no_text = test_df.drop(columns=['text','keyword']).join(test_tfidf)

In [155]:
#Agrego las otras predicciones como features
pred_train_catboost=pd.read_csv('feature_pred/prediccion_train_catBoost.csv')
pred_test_catboost=pd.read_csv('feature_pred/prediccion_test_catBoost.csv')
pred_train_bayes=pd.read_csv('feature_pred/prediccion_train_bayes.csv')
pred_test_bayes=pd.read_csv('feature_pred/prediccion_test_bayes.csv')
pred_train_svm=pd.read_csv('feature_pred/prediccion_train_SVM.csv')
pred_test_svm=pd.read_csv('feature_pred/prediccion_test_SVM.csv')

In [156]:
train_no_text['pred_catbooost'] = pred_train_catboost.target
test_no_text['pred_catbooost'] = pred_test_catboost.target

train_no_text['pred_bayes'] = pred_train_bayes.target
test_no_text['pred_bayes'] = pred_test_bayes.target

train_no_text['pred_svm'] = pred_train_svm.target
test_no_text['pred_svm'] = pred_test_svm.target


In [157]:
cs = MinMaxScaler()

In [158]:
trainContinuous = cs.fit_transform(train_no_text)
testContinuous = cs.fit_transform(test_no_text)

Primero pruebo sin keywords

In [159]:
kf = KFold(n_splits = 5)
kf

KFold(n_splits=5, random_state=None, shuffle=False)

In [160]:
nlp_text_input = keras.Input(shape=(180,), name='nlp_text_input')
meta_input = keras.Input(shape=(train_no_text.shape[1],), name='meta_input')

In [163]:
test_error=[]
#Si agregamdp una dense mejora el resultado
for train_index,test_index in kf.split(train_data):
    x_train_words,x_test_words = train_data[train_index], train_data[test_index]
    x_train_numerical,x_test_numerical = trainContinuous[train_index], trainContinuous[test_index]
    y_train,y_test = train_df['target'][train_index], train_df['target'][test_index]
    
    emb = Embedding(50000, 200, input_length=100, weights=[embedding_matrix], trainable=False)(nlp_text_input)
    nlp_text_out = Bidirectional(LSTM(200, dropout=0.6, recurrent_dropout=0.2))(emb)  #conv antes era emb


    x = concatenate([nlp_text_out ,meta_input])
    x = Dense(24, activation='relu')(x)
    x = Dense(1, activation='sigmoid')(x)

    model = Model(inputs=[nlp_text_input , meta_input], outputs=[x])
    model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])

    model.fit([x_train_words, x_train_numerical], y_train,validation_data=([x_test_words,x_test_numerical],y_test), batch_size=40, epochs=5,verbose=2)

    predictions= model.predict([x_test_words,x_test_numerical])

    bin_pred = []
    for i in predictions:
        if (i<0.46):
            bin_pred.append(0)
        else:
            bin_pred.append(1)

    test_error.append(f1_score(y_test, bin_pred, average='macro'))
    
print('Error de test:',test_error)

Epoch 1/5
153/153 - 63s - loss: 0.5092 - accuracy: 0.7634 - val_loss: 0.4204 - val_accuracy: 0.8168
Epoch 2/5
153/153 - 67s - loss: 0.4498 - accuracy: 0.7970 - val_loss: 0.4222 - val_accuracy: 0.8116
Epoch 3/5
153/153 - 60s - loss: 0.4267 - accuracy: 0.8082 - val_loss: 0.4214 - val_accuracy: 0.8148
Epoch 4/5
153/153 - 61s - loss: 0.4102 - accuracy: 0.8135 - val_loss: 0.4254 - val_accuracy: 0.8175
Epoch 5/5
153/153 - 59s - loss: 0.4001 - accuracy: 0.8232 - val_loss: 0.4263 - val_accuracy: 0.8148
Epoch 1/5
153/153 - 61s - loss: 0.5094 - accuracy: 0.7562 - val_loss: 0.4672 - val_accuracy: 0.7892
Epoch 2/5
153/153 - 64s - loss: 0.4303 - accuracy: 0.8066 - val_loss: 0.4699 - val_accuracy: 0.7912
Epoch 3/5
153/153 - 56s - loss: 0.4152 - accuracy: 0.8123 - val_loss: 0.4677 - val_accuracy: 0.7886
Epoch 4/5
153/153 - 56s - loss: 0.4076 - accuracy: 0.8163 - val_loss: 0.4655 - val_accuracy: 0.7899
Epoch 5/5
153/153 - 52s - loss: 0.3897 - accuracy: 0.8307 - val_loss: 0.4980 - val_accuracy: 0.7695


In [164]:
predictions= model.predict([test_data,testContinuous])
bin_pred = []
bin_probably_wrong = []
for i in predictions:
    if (i<0.46): #Important
        bin_pred.append(0)
    else:
        bin_pred.append(1)
        
for j in predictions:
    if (j<0.55 and j > 0.45):
        bin_probably_wrong.append(2)

In [165]:
sample.target = bin_pred

In [166]:
sample.target.value_counts()

0    2060
1    1203
Name: target, dtype: int64

In [167]:
sample.to_csv('NN-tfidf-predictions-5epochs.csv', index=False)

In [168]:
predictions = model.predict([x_test_words,x_test_numerical])

In [169]:
prediccion_train_spliteada = train_no_text.loc[test_index]

In [170]:
prediccion_train_spliteada['tensor_flow_pred'] = predictions

In [171]:
prediccion_train_spliteada['target'] = train_df['target'][test_index]

In [172]:
prediccion_train_spliteada.to_csv('split_train_pred_tensor_flow.csv',index=False)

In [173]:
predictions= model.predict([test_data,testContinuous])

In [174]:
test_no_text['tensor_flow_pred'] = predictions

In [175]:
test_no_text[['tensor_flow_pred']].to_csv('test_pred_tensor_flow.csv',index=False)